In [38]:
import requests
import json
import pandas as pd

In [39]:
#Here are a few coordinates, and you have to find the fortnightly average temperature for these coordinates(longitude and latitude):
lat_long_deg_min_sec = [
('37° 52\' 5.336" S', '144° 55\' 40.452" E'),
('38° 19\' 57.125" S', '144° 54\' 17.831" E'),
('38° 51\' 34.878" S', '142° 54\' 18.281" E'), 
('37° 50\' 33.481" S', '135° 38\' 20.388" E'),
('35° 5\' 50.501" S', '128° 48\' 40.243" E'), 
('34° 20\' 7.398" S', '121° 35\' 19.757" E'),
('34° 55\' 34.99" S', '118° 45\' 4.596" E'), 
('35° 3\' 36.441" S', '115° 51\' 54.355" E'),
('30° 45\' 16.173" S', '110° 26\' 25.532" E'),
('12° 5\' 45.683" S', '106° 25\' 8.984" E'),
('6° 18\' 54.271" S', '105° 33\' 56.07" E'), 
('5° 15\' 14.261" S', '106° 13\' 37.411" E'),
('3° 42\' 37.526" S', '107° 35\' 13.608" E'), 
('2° 0\' 50.122" S', '107° 12\' 50.324" E'),
('0° 42\' 51.809" N', '105° 19\' 28.118" E'), 
('1° 18\' 56.371" N', '104° 19\' 31.582" E'),
('1° 13\' 14.328" N', '103° 52\' 6.734" E')
]

In [40]:
import re
def convert(tude):
    multiplier = 1 if tude[-1] in ['N', 'E'] else -1
    tude = re.split('° |\' |" ', tude)
    #print(tude)
    return multiplier * sum(float(x) / 60 ** n for n, x in enumerate(tude[:-1]))

In [41]:
lat_long_dec = []

for lat_long in lat_long_deg_min_sec:
    lat = convert(lat_long[0])
    long = convert(lat_long[1])
    lat_long_dec.append((lat, long))

print(lat_long_dec)

[(-37.86814888888889, 144.92790333333332), (-38.33253472222223, 144.90495305555555), (-38.85968833333334, 142.90507805555555), (-37.84263361111111, 135.63899666666666), (-35.09736138888889, 128.81117861111113), (-34.335388333333334, 121.58882138888889), (-34.92638611111111, 118.75127666666667), (-35.0601225, 115.86509861111111), (-30.7544925, 110.44042555555556), (-12.096023055555555, 106.41916222222223), (-6.315075277777778, 105.565575), (-5.2539613888888885, 106.22705861111112), (-3.710423888888889, 107.58711333333333), (-2.0139227777777777, 107.21397888888889), (0.7143913888888889, 105.32447722222221), (1.3156586111111113, 104.32543944444444), (1.2206466666666669, 103.86853722222222)]


In [42]:
CLIENT_ID = '1edaebb5332c44b8ac3124613212406'

for lat, long in lat_long_dec:
    print(lat, long, '-----------------------------------------------------------')

    # create the API request URL
    url = 'https://api.worldweatheronline.com/premium/v1/weather.ashx?key={}&q={},{}&format=json&num_of_days={}'.format(
        CLIENT_ID,
        lat, 
        long, 
        5)
    
    #response  = requests.get(url)
    #forecasts  = response.json()
    
    try:
        print("Date: {}, Max Temperature: {}".format(forecasts['data']['weather'][0]['date'], forecasts['data']['weather'][0]['maxtempC']))
    except:
        print('Error: {}'.format(forecasts['data']['error'][0]))
    

-37.86814888888889 144.92790333333332 -----------------------------------------------------------
Error: {'msg': 'API key has reached calls per day allowed limit.'}
-38.33253472222223 144.90495305555555 -----------------------------------------------------------
Error: {'msg': 'API key has reached calls per day allowed limit.'}
-38.85968833333334 142.90507805555555 -----------------------------------------------------------
Error: {'msg': 'API key has reached calls per day allowed limit.'}
-37.84263361111111 135.63899666666666 -----------------------------------------------------------
Error: {'msg': 'API key has reached calls per day allowed limit.'}
-35.09736138888889 128.81117861111113 -----------------------------------------------------------
Error: {'msg': 'API key has reached calls per day allowed limit.'}
-34.335388333333334 121.58882138888889 -----------------------------------------------------------
Error: {'msg': 'API key has reached calls per day allowed limit.'}
-34.92638

In [43]:
def get_wwo_historical(lat, long, start_date, end_date):
    CLIENT_ID = '1edaebb5332c44b8ac3124613212406'
    
    # create the API request URL
    url = 'https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={}&q={},{}&format=json&date={}&enddate={}&tp=24'.format(
        CLIENT_ID,
        lat, 
        long, 
        start_date,
        end_date)
    
    response  = requests.get(url)
    forecasts  = response.json()
    
    try:
        print("Date: {}, Max Temperature: {}".format(forecasts['data']['weather'][0]['date'], forecasts['data']['weather'][0]['maxtempC']))
        return forecasts['data']['weather']
    except:
        print('Error: {}'.format(forecasts['data']['error'][0]))

In [45]:
!pip install wwo-hist

In [46]:
from wwo_hist import retrieve_hist_data

In [55]:
frequency=24
start_date = '1-JUN-2019'
end_date = '1-JUN-2021'
api_key = CLIENT_ID


hist_weather_data_df = pd.DataFrame()

for lat, long in lat_long_dec:
    location_list = []
    
    location_list.append('{},{}'.format(lat, long))
    
    try:
        hist_weather_data_list = retrieve_hist_data(api_key,
                                    location_list,
                                    start_date,
                                    end_date,
                                    frequency,
                                    location_label = False,
                                    export_csv = False,
                                    store_df = True)
        
        # hist_weather_data_list is a list of datframes. Since we are making one call for one location, we get the first dataframe as the first element of the list
        hist_weather_data_df = hist_weather_data_df.append(hist_weather_data_list[0])
    
    except Exception as e:
        print(e)
        print('Unable to find any matching weather for location {}'.format(location_list))




Retrieving weather data for -37.86814888888889,144.92790333333332


Currently retrieving data for -37.86814888888889,144.92790333333332: from 2019-06-01 to 2019-06-30
Time elapsed (hh:mm:ss.ms) 0:00:00.779627
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2019-07-01 to 2019-07-31
Time elapsed (hh:mm:ss.ms) 0:00:01.670891
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2019-08-01 to 2019-08-31
Time elapsed (hh:mm:ss.ms) 0:00:02.676461
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2019-09-01 to 2019-09-30
Time elapsed (hh:mm:ss.ms) 0:00:03.519447
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2019-10-01 to 2019-10-31
Time elapsed (hh:mm:ss.ms) 0:00:04.399305
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2019-11-01 to 2019-11-30
Time elapsed (hh:mm:ss.ms) 0:00:05.269434
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2019-12-01

In [56]:
hist_weather_data_df['date_time'] = pd.to_datetime(hist_weather_data_df['date_time'])
hist_weather_data_df['tempC'] = hist_weather_data_df['tempC'].astype('int')

In [57]:
hist_weather_data_df.reset_index(inplace=True, drop=True)
hist_weather_data_df

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2019-06-01,13,13,0.0,3.7,3,0,04:41 AM,03:56 PM,07:27 AM,...,18,86,80,2.5,1031,13,7,196,13,"-37.86814888888889,144.92790333333332"
1,2019-06-02,15,10,0.0,7.0,3,0,05:44 AM,04:30 PM,07:27 AM,...,21,71,80,16.3,1025,15,7,266,15,"-37.86814888888889,144.92790333333332"
2,2019-06-03,12,10,0.0,3.7,3,0,06:49 AM,05:11 PM,07:28 AM,...,50,81,80,29.9,1021,12,8,184,30,"-37.86814888888889,144.92790333333332"
3,2019-06-04,12,10,0.0,5.4,3,6,07:55 AM,05:58 PM,07:29 AM,...,30,77,68,1.9,1028,12,9,170,21,"-37.86814888888889,144.92790333333332"
4,2019-06-05,12,11,0.0,3.7,3,13,08:59 AM,06:53 PM,07:29 AM,...,24,86,73,1.8,1031,12,10,224,17,"-37.86814888888889,144.92790333333332"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10243,2021-05-28,31,28,0.0,9.2,6,83,08:12 PM,07:34 AM,05:56 AM,...,8,69,72,2.4,1010,31,10,163,6,"1.2206466666666669,103.86853722222222"
10244,2021-05-29,32,28,0.0,10.5,7,76,09:15 PM,08:37 AM,05:56 AM,...,12,56,69,1.7,1009,32,10,173,9,"1.2206466666666669,103.86853722222222"
10245,2021-05-30,30,28,0.0,7.8,6,69,10:14 PM,09:39 AM,05:57 AM,...,22,62,73,14.1,1009,30,9,150,14,"1.2206466666666669,103.86853722222222"
10246,2021-05-31,31,27,0.0,7.8,6,65,11:08 PM,10:36 AM,05:57 AM,...,19,69,74,9.8,1007,31,9,157,13,"1.2206466666666669,103.86853722222222"


In [58]:
grouped = hist_weather_data_df.groupby('location')
grouped

In [59]:
!pip install folium

In [60]:
!pip install vincent

In [61]:
import folium
import vincent

# To initialize vincent in the notebook
vincent.core.initialize_notebook()

In [62]:
center_lat = sum(value[0] for value in lat_long_dec) / len(lat_long_dec)
center_long = sum(value[1] for value in lat_long_dec) / len(lat_long_dec)

m = folium.Map(location=[center_lat, center_long], zoom_start=4)

for name, group in grouped:
    print(name)
    
    loc_df = group[['date_time', 'tempC']].set_index('date_time')
    
    loc_df = loc_df.resample('2W').mean()
    
    five_years_line = vincent.Line(loc_df, width=300, height=150)
    five_years_line.axis_titles(x='Date', y='tempC')
    #five_years_line.display()
    
    five_years_vega = folium.Vega(five_years_line.to_json(), width=350, height=200)
    
    lat = name.split(',')[0]
    long = name.split(',')[1]
    
    marker = folium.Marker(
            location = [lat, long],
            fill_opacity = 0.8, color=None,
            fill_color = ('#274cc9'),
            popup = folium.Popup().add_child(five_years_vega),
            tooltip = '{}, {}'.format(lat, long)
            )

    marker.add_to(m)

m

-2.0139227777777777,107.21397888888889
-3.710423888888889,107.58711333333333
-34.335388333333334,121.58882138888889
-34.92638611111111,118.75127666666667
-35.0601225,115.86509861111111
-37.84263361111111,135.63899666666666
-37.86814888888889,144.92790333333332
-38.33253472222223,144.90495305555555
-38.85968833333334,142.90507805555555
-5.2539613888888885,106.22705861111112
-6.315075277777778,105.565575
0.7143913888888889,105.32447722222221
1.2206466666666669,103.86853722222222
1.3156586111111113,104.32543944444444
